In [4]:
import cv2
import numpy as np
import random
import nest_asyncio
from flask import Flask, render_template, Response
import os

# Fix Flask for Jupyter Notebook
nest_asyncio.apply()

app = Flask(__name__)

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load ghost images (Ensure they are in static/ghosts/)
ghost_images = [
    cv2.imread("static/ghosts/ghost_1.jpeg"),
    cv2.imread("static/ghosts/ghost_2.jpeg"),
    cv2.imread("static/ghosts/ghost_3.jpg")
]

def detect_face_tilt(faces, frame_width):
    """Detect if the face is tilted left or right."""
    if len(faces) == 1:
        x, y, w, h = faces[0]
        face_center_x = x + w // 2  # Find the center of the detected face

        if face_center_x < frame_width * 0.4:  # Tilted Left
            return "left"
        elif face_center_x > frame_width * 0.6:  # Tilted Right
            return "right"
    
    return "straight"  # If no face detected or in center, return 'straight'

def overlay_ghost(frame, ghost):
    """Overlay a ghost image onto the frame."""
    ghost_resized = cv2.resize(ghost, (frame.shape[1], frame.shape[0]))  # Resize ghost to fit full frame
    alpha = 1  # Adjust transparency level (0.0 - 1.0)
    return cv2.addWeighted(frame, 1, ghost_resized, alpha, 0)

def generate_frames():
    cap = cv2.VideoCapture(0)  # Open webcam

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Mirror effect
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        frame_width = frame.shape[1]
        face_tilt = detect_face_tilt(faces, frame_width)

        if face_tilt in ["left", "right"]:
            ghost = random.choice(ghost_images)  # Randomly choose a ghost image
            if ghost is not None:
                frame = overlay_ghost(frame, ghost)

        _, buffer = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Mar/2025 15:02:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2025 15:02:04] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2025 15:02:04] "GET /static/ghosts/ghost_2.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2025 15:02:04] "GET /mirror_state HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2025 15:02:04] "GET /static/sounds/ghost_sound.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [01/Mar/2025 15:02:07] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2025 15:02:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2025 15:02:10] "GET /static/sounds/ghost_sound.mp3 HTTP/1.1" 206 -
